<a href="https://colab.research.google.com/github/PStettler/Schneider-Group-DCBP/blob/main/restrictionEnyzmeMotifFinder%26Fixer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>Welcome to the Restriction Enyzme Motif finder and fixer of the Schneider Group!</h2>

This tool locates Restriction enzyme motifs for you and suggests changes of the sequence to fix them! Amazing no? Say no more to waking up at night from dreeming of two bands on your agarose gel!

You can provide you sequence below. There is a hard-coded list of restriction enzyme motifs that we work with frequently. If you are looking for some more fancy ones, you can provide them too down below. Have fun!

In [3]:
sequence = 
  #please replace the example DNA sequence below with your sequence of interest, use 
  #only DNA characters (ATGCNRYSWKMBDHV) in capital letters. 
  "
      ATGAGCGGCCGCGCAGCGGCCGCTGATTTCTTCAACATCGTCGACGACTTCCTGAAGAAAACTTTCAGAGATGAGTCGTT
      TCTATTCGCTGCAGTCAAGTCCAGGCAGTATTCCGAGTCTTTTCCGGGTGAACAACTCTTCTTCACCTCCCCGCCTTCGG
      CGACGACTGACCAGCCCTCGGATGAGAGCCTTCAAGCAGGAGGAGGCGAGTTGAGGAAAACGCAAAATTTTATGTACTTC
      TCGCCCCGTCTAATTTTCAACAGAGACGGAGGGTACCATGGGAAGGTGAAGTTGCACAGTGGTGTTCACGTGCCGCAGGT
      GTGTCGGTTTGAGCAAGGGATTGCTGTGAACAGCGAGGGGTTGGCATCGGGTAGCGTCAAGGTTAGTGATCTTCTTGAGG
      GAATGGAGGTAAAAGGGCGTCTCGCAGTAAACACAATTGCTCCTCCGTCAAAAGATGTGTGGTCCGTTGCAATGGATTAC
      CAGCGTCGTGACTTCTATTCCACACTTAACTATCAGAGGAACGGATTGGGAAGCAGTGACTTGCTTGTTGATTGTGGCAC
      GAAATTTTTTAATCTTCTTGCGGGTGCCGGATTTGAGCGGCAGAAAGTTTCGTTTCTTGAGCAACAGGATCACACCGCGC
      AGTTGGACGTGCTTTATGCGGGGGTTGGCTTTACTGGGGTGAATTGGTCCGTTGGTGCAAAGTTAGTGAGAGCGAACGAT
      ATGTGGAGCGCGGCACGCATTGCTTTTTATCAGCGCGTGGTGCCCGACACATCGGTGGCCTGCGCATACAATTTCGATAT
      GGAGGAGTCACGCGTACATGTCTCGCTGGGATTTTCACAGGGTTTTCGGCTTCGCGTCCCCACGATTCTGCAGCAGCGGG
      CGTGTGAGCAGTTGGACGTATGGACCGCCATTCTCCCGTTTGTGGGAGCATTCAAGGTGGAGAGCGGTGGCTTATGTGCG
      GCAACCATTCGTGGCATATTTAATGGCGTGGTGCATTGGGGTTTGGTGGCGCAGAAGAACGTGTTGGTGGAGAACAGTCC
      CATCCGTTTCGGTCTCACTCTTTCCGTGGAATCCGGTTGA
  
  "

user.enzyme.sites = 
  #here you can add more enzymes to the standard list that is tested 
  #(the standard list can be seen below)
  #do it as follow: name: Motif & name2: motif2 & etc.
  #for example: EcoRI: "GAATTC  & SacI: GAGCTC"
  #write this between the partentheses below. 
  #If you don't want to add any, just leave it as is.
  "     
  
  
  
  "

  #Now go to Runtime, and select run everything. The results will be displayed below 
  #this box. Have fun.

This is the list of all presaved restriction enzyme cut sites:

NotI    GCGGCCGC

HindIII   AAGCTT

BamHI     GGATCC

XhoI      CTCGAG

XbaI      TCTAGA

NdeI      CATATG

SacI      GAGCTC


In [ ]:
#@title Standardtext für Titel
#################################################################
##########Restrictione enzyme site finder and fixer##############
################Philip Stetter, 19.03.2023#######################
#################################################################

#This tool locates undesired restrictione enzyme cut sites
#in your gene/sequence of interest and suggests modifications
#to remove these cut sites. You are welcome.

require(stringr)

###########################################################################
#hard-coding of the custumizable retrictione enzyme list
#here, a pre-coded list ist established. The user can provide
#additional enzyme sites that will be appended to this list.
restriction.enzymes = c("NotI"    = "GCGGCCGC",
                        "HindIII" = "AAGCTT",
                        "BamHI"   = "GGATCC",
                        "XhoI"    = "CTCGAG",
                        "XbaI"    = "TCTAGA",
                        "NdeI"    = "CATATG",
                        "SacI"    = "GAGCTC")

###########################################################################
#the used can provide more sites to be tested, write a function that can do this
#user.enzyme.sites = "      EcoRI: GAATTC  & sacI: GAGCTC" #example of the user input
#pattern has to be name: mitof, several have to be separated by &

append.enzyme.list = function(current.list,user.input){
  user.input.no.whitespace = paste(unlist(str_extract_all(user.input,"[A-Za-z0-9:&]")),sep="",collapse="")
  namesAndSequence = unlist(str_split(user.input.no.whitespace,"[&]"))
  names.new = NA
  motif.new = NA
  for(i in 1:length(namesAndSequence)){
    names.new[i] = unlist(str_split(namesAndSequence[i],":"))[1]
    motif.new[i] = unlist(str_split(namesAndSequence[i],":"))[2]
  }
  if(!is.na(names.new[1])){
    new.list = c(current.list,motif.new)
    names(new.list) = c(names(current.list),names.new)
  }
  return(new.list)
}

####################################################################

#sequence of interest provided by the user

sequence = str_replace_all(sequence,"[^ATGCNRYSWKMBDHV]","")


########################################################################


#a function to check if the sequence has an open reading 
#frame and return the start and stop nucleotide number.
has_ORF= function(sequence){
  if(str_detect(sequence,"ATG([ATGC]{3})*((TGA)|(TAA)|(TAG))"))
  {
    #detect open reading frame (ORF)
    regex = "ATG([ATGC]{3})+(?=((TGA)|(TAA)|(TAG)))"
    ORF.temp = sequence
    while(str_detect(ORF.temp,regex)){
      #a while loop to detect the correct open reading frame
      #starting at the first atg and ending at the closest inframe stop codon
      ORF.temp = str_extract(ORF.temp,regex)
      if(!str_detect(regex,"\\^")) {regex = paste0("^",regex)}
    }
    stop.codon = str_extract(sequence,paste0("(?<=",ORF.temp,")((TGA)|(TAA)|(TAG))"))
    ORF = paste0(ORF.temp,stop.codon)
    # cat("an open reading frame was detected, the sequence is:\n",ORF,"\n\n")
    return(list("ORF_status" = TRUE, "location" = str_locate(sequence,ORF)))
    
  }else{return(list("ORF_status" = FALSE, "location" = NA))}
}
# has_ORF(sequence)
if(has_ORF(sequence)$ORF_status){
  ORF_start = has_ORF(sequence)$location[1,1]
  ORF_end = has_ORF(sequence)$location[1,2]
}else{cat("The sequence has no open reading frame, restriction enzyme sites will only be searches but not fixed.")}


sequence.codons = unlist(str_extract_all(substr(sequence,ORF_start,ORF_end),"[ATGCNRYSWKMBDHV]{3}"))

######################################################################
#load a codon table
{ #load codon table
  codons.1 = c(rep("T",16),rep("C",16),rep("A",16),rep("G",16))
  codons.2 = rep(c(rep("T",4),rep("C",4),rep("A",4),rep("G",4)),4)
  codons.3 = rep(c("T","C","A","G"),16)
  one.letter.not = c("F","F","L","L","S","S","S","S","Y","Y","*","*","C","C","*","W",
                     "L","L","L","L","P","P","P","P","H","H","Q","Q","R","R","R","R",
                     "I","I","I","M","T","T","T","T","N","N","K","K","S","S","R","R",
                     "V","V","V","V","A","A","A","A","D","D","E","E","G","G","G","G")
  codons = cbind(paste(codons.1,codons.2,codons.3,sep=""),one.letter.not)
}#load codon table end

#load T. brucei codon usage 
{ #codon usage table of T. brucei from kazusa.org
  # UUU F 0.56 20.5 ( 53927)  UCU S 0.16 12.6 ( 33149)  UAU Y 0.44 11.3 ( 29639)  UGU C 0.49 10.9 ( 28666)
  # UUC F 0.44 15.9 ( 41800)  UCC S 0.16 12.5 ( 32896)  UAC Y 0.56 14.1 ( 37043)  UGC C 0.51 11.3 ( 29707)
  # UUA L 0.10  9.8 ( 25683)  UCA S 0.17 13.5 ( 35511)  UAA * 0.36  0.7 (  1874)  UGA * 0.36  0.7 (  1875)
  # UUG L 0.21 19.5 ( 51383)  UCG S 0.15 11.5 ( 30350)  UAG * 0.27  0.5 (  1389)  UGG W 1.00 10.9 ( 28786)
  # 
  # CUU L 0.24 22.3 ( 58573)  CCU P 0.23 11.1 ( 29130)  CAU H 0.47 11.3 ( 29758)  CGU R 0.23 15.9 ( 41800)
  # CUC L 0.17 15.6 ( 40909)  CCC P 0.23 11.1 ( 29151)  CAC H 0.53 13.0 ( 34152)  CGC R 0.21 14.5 ( 38107)
  # CUA L 0.09  8.3 ( 21743)  CCA P 0.29 13.9 ( 36504)  CAA Q 0.45 16.9 ( 44506)  CGA R 0.13  9.0 ( 23569)
  # CUG L 0.20 18.5 ( 48568)  CCG P 0.25 11.8 ( 30918)  CAG Q 0.55 21.0 ( 55275)  CGG R 0.18 12.1 ( 31803)
  # 
  # AUU I 0.47 19.0 ( 49922)  ACU T 0.23 13.0 ( 34212)  AAU N 0.49 18.2 ( 47864)  AGU S 0.19 15.1 ( 39666)
  # AUC I 0.29 11.6 ( 30419)  ACC T 0.21 12.1 ( 31844)  AAC N 0.51 19.2 ( 50557)  AGC S 0.17 13.6 ( 35753)
  # AUA I 0.25 10.0 ( 26313)  ACA T 0.30 17.5 ( 45955)  AAA K 0.44 20.8 ( 54602)  AGA R 0.10  6.8 ( 17992)
  # AUG M 1.00 23.4 ( 61545)  ACG T 0.26 14.8 ( 39063)  AAG K 0.56 26.6 ( 69988)  AGG R 0.15 10.0 ( 26246)
  # 
  # GUU V 0.30 22.9 ( 60206)  GCU A 0.25 20.8 ( 54709)  GAU D 0.55 28.1 ( 73880)  GGU G 0.34 22.6 ( 59534)
  # GUC V 0.15 11.4 ( 29932)  GCC A 0.22 18.3 ( 48021)  GAC D 0.45 22.7 ( 59712)  GGC G 0.22 14.9 ( 39184)
  # GUA V 0.17 12.6 ( 33047)  GCA A 0.28 23.3 ( 61330)  GAA E 0.46 31.7 ( 83517)  GGA G 0.23 15.6 ( 41118)
  # GUG V 0.38 28.6 ( 75313)  GCG A 0.25 20.7 ( 54480)  GAG E 0.54 38.0 ( 99928)  GGG G 0.21 13.9 ( 36525)
  t.brucei.codon.usage.fraction = list(c(0.56,0.44),c(0.1,0.21,0.24,0.17,0.09,0.2),c(0.16,0.16,0.17,0.15,0.19,0.17),
                                       c(0.44,0.56),c(0.36,0.28,0.36),c(0.49,0.51),c(1),c(0.23,0.23,0.29,0.25),
                                       c(0.47,0.53),c(0.45,0.55),c(0.23,0.21,0.13,0.18,0.10,0.15),c(0.46,0.29,0.25),
                                       c(1),c(0.23,0.21,0.3,0.26),c(0.49,0.51),c(0.44,0.56),c(0.3,0.15,0.17,0.38),
                                       c(0.25,0.22,0.28,0.25),c(0.55,0.45),c(0.46,0.54),c(0.34,0.22,0.23,0.21))
  t.brucei.codon.usage.codon = list(c("TTT","TTC"),c("TTA","TTG","CTT","CTC","CTA","CTG"),c("TCT","TCC","TCA","TCG","AGT","AGC"),
                                    c("TAT","TAC"),c("TAA","TAG","TGA"),c("TGT","TGC"),c("TGG"),c("CCT","CCC","CCA","CCG"),
                                    c("CAT","CAC"),c("CAA","CAG"),c("CGT","CGC","CGA","CGG","AGA","AGG"),c("ATT","ATC","ATA"),
                                    c("ATG"),c("ACT","ACC","ACA","ACG"),c("AAT","AAC"),c("AAA","AAG"),c("GTT","GTC","GTA","GTG"),
                                    c("GCT","GCC","GCA","GCG"),c("GAT","GAC"),c("GAA","GAG"),c("GGT","GGC","GGA","GGG"))
  names(t.brucei.codon.usage.fraction) = c("F","L","S","Y","*","C","W","P","H","Q","R","I","M","T","N","K","V","A","D","E","G")
  names(t.brucei.codon.usage.codon) = c("F","L","S","Y","*","C","W","P","H","Q","R","I","M","T","N","K","V","A","D","E","G")
} #fuck me


#############################################################################################


#getting started, analyze the provided sequence
#update the restriction enzyme list (if given)
if(str_detect(user.enzyme.sites,"[A-Za-z0-9:&]")){
restriction.enzymes = append.enzyme.list(restriction.enzymes,user.enzyme.sites)
}

#locate all restrictione enzyme cut sites in the user sequence
sites.in.sequence = NA ; j=1
sites.not.in.sequence = NA ; k=1
for(i in 1:length(restriction.enzymes)){
  if(str_detect(sequence,restriction.enzymes[i])){
    sites.in.sequence[j] = restriction.enzymes[i]
    names(sites.in.sequence)[j] = names(restriction.enzymes)[i]
    j = j+1
  }else{
    sites.not.in.sequence[k] = restriction.enzymes[i]
    names(sites.not.in.sequence)[k] = names(restriction.enzymes)[i]
    k = k+1
  }
}
if(j==1){cat("The provided sequence does not contain any of the tested Restriction enzyme cut sites.",
            "\n\n The following enzymes were tested:\n\n");print(data.frame(motif=restriction.enzymes))}

if(!is.na(sites.in.sequence[1])){
#locate the sites that are in the sequence
sites.location.start = NA
sites.location.end = NA
sites.location.withinORF = NA
sites.location.sequence = NA
k = 1
for(i in 1:length(sites.in.sequence)){
  #count how often the site occurs
  count = str_count(sequence,sites.in.sequence[i])
  for(j in 1:count){
    #retrieve all locations and check if they are within the ORF
    sites.location.start[k] = str_locate_all(sequence,sites.in.sequence[i])[[1]][j,1]
    sites.location.end[k] = str_locate_all(sequence,sites.in.sequence[i])[[1]][j,2]
    sites.location.withinORF[k] = ifelse(sites.location.start[k] <= ORF_end & 
                                           sites.location.end[k] >= ORF_start,
                                         TRUE,FALSE)
    sites.location.sequence[k] = sites.in.sequence[i]
    names(sites.location.start)[k] = paste0(names(sites.in.sequence)[i],"#",j)
    names(sites.location.end)[k] = paste0(names(sites.in.sequence)[i],"#",j)
    names(sites.location.withinORF)[k] = paste0(names(sites.in.sequence)[i],"#",j)
    k = k+1
  }
}


#print all results to console
if(has_ORF(sequence)$ORF_status){
cat("An open reading frame (OFR) was detected in the sequence, it starts at nucleotide ", ORF_start,
    " and ends at nucleotide ", ORF_end,".\n\n")
}
cat("The following restriction enzyme sites were detected in the provided sequence \n (the # indicates how", 
    "many times the site was found)\n\n")
print(data.frame("start nucleotide" = sites.location.start,
                 "end nucleotide" = sites.location.end))
cat("\n\n This is the list of all tested restriction enzyme cut sites:\n\n")
print(data.frame(motif=restriction.enzymes))



#lets move on with fixing the sites
has_restriction_enzyme_sites = sum(str_detect(sequence,restriction.enzymes))>0

runs=0 #there is a risk of not finding a solution, se we shall use a break point

set.seed(1)

#fix the restriction enzyme cut sites
if(has_ORF(sequence)$ORF_status){
  
  while(has_restriction_enzyme_sites | runs > 1000){
    
  #go through the list of detected sites
  #and try to fix them
  sequence.modified = sequence
  
  for(i in 1:length(sites.location.withinORF)){
    if(!sites.location.withinORF[i]){cat("The site of ",names(sites.location.withinORF)[i],
                                         " was not tried to be fixed as it is outside the ORF")
    }else{
      #calculate in which codons the site is
      if(sites.location.start[i] >= ORF_start){#site must start in a codon
        codon.site.start = ceiling(((sites.location.start[i] - ORF_start) +1)/3)
        codon.site.end = ceiling(((sites.location.end[i] - ORF_start) +1)/3)
        #site could end outside ORF, check and correct if neccessary
        codon.site.end = ifelse(codon.site.end > length(sequence.codons),length(sequence.codons),codon.site.end)
        
      }else{#site must end in the ORF
        codon.site.start = 1
        codon.site.end = ceiling(((sites.location.end[i] - ORF_start) +1)/3)
      }
      
      
      sequence.modified.test = sequence.modified
      while(sum(str_detect(substr(sequence.modified.test,sites.location.start[i],sites.location.end[i]),
                           restriction.enzymes))>0){
        #always re-set the test sequence to the original modified one
        #so that unneccessary changes are not saved
        squence.modified.test = sequence.modified
        
        #modify the codons in the loop until no sites are present anymore
        #this loop will only remove one site at a time within the for loop
        #over all sites. Since next to the changed sites new sites might appear,
        #everything is run within another while loop.
        #ramdomly select one codon and change it
        codon.to.change = sample(codon.site.start:codon.site.end,1)
        aa = codons[which(sequence.codons[codon.to.change]==codons[,1]),2]
        new.tripplets = sample(unlist(t.brucei.codon.usage.codon[aa]),
                               1,
                               prob = unlist(t.brucei.codon.usage.fraction[aa]),
                               replace = TRUE)
        #replace the codon
        substr(sequence.modified.test,(codon.to.change-1)*3 +1,(codon.to.change-1)*3 +1 +2) = new.tripplets
      }
      sequence.modified = sequence.modified.test #the site was fixed, so save and move on
    }
  }
  
  runs = runs +1
  
  has_restriction_enzyme_sites = sum(str_detect(sequence.modified,restriction.enzymes))>0
  
  }
  
}
#exit due to no solution?
if(runs>1000){cat("no solution to remove all sites was found. sorry.")
}else{
  #pretty print the solution  
  
  track.of.changes = unlist(str_split(sequence.modified,""))
  track.of.changes[track.of.changes==unlist(str_split(sequence,""))] = "|"
  sites.on.sequence = rep(" ",nchar(sequence))
  for(i in 1:length(sites.location.start)){
    sites.on.sequence[sites.location.start[i]:(sites.location.start[i]+nchar(names(sites.location.start)[i])-1)] = 
      unlist(str_split(names(sites.location.start)[i],""))
  }
  sites.on.sequence = paste(sites.on.sequence,sep="",collapse="")
  sequence = paste(sequence,sep="",collapse="")
  track.of.changes = paste(track.of.changes,sep="",collapse="")
  sequence.modified = paste(sequence.modified,sep="",collapse="")
  
  cat("\n\nThis is the overview of the suggested changes:\n",
      "(1: detected enzyme sites, 2: input sequence, 3: modifications to remove motifs, 4: suggested sequence)\n")
  l = 1
  continue.print = TRUE
  while(continue.print){
    if(l==nchar(sequence)){continue.print = FALSE}
    else{sites.on.sequence.out = substr(sites.on.sequence,l,l+99)
    sequence.out = substr(sequence,l,l+99)
    track.of.changes.out = substr(track.of.changes,l,l+99)
    sequence.modified.out = substr(sequence.modified,l,l+99)
    print(c(sites.on.sequence.out,sequence.out,track.of.changes.out,sequence.modified.out))
    l = l+100
    l = ifelse(l<=nchar(sequence),l,nchar(sequence))
    cat("\n")
    }
  }
  
  cat("This is the final result (to be copy and pasted):\n\n")
  print(sequence.modified)
  
  
  }


}

#end of script